In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, when, split, col, lit, max, min, expr
from pyspark.sql.functions import to_date, var_samp, variance, var_pop, month, to_timestamp, dayofweek
from pyspark.sql.types import NumericType, IntegerType, FloatType

In [2]:
spark = SparkSession.builder \
    .appName("Maestria_evidencia1") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "32g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.default.parallelism", "16") \
    .config("spark.driver.maxResultSize", "16g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Mejores tablas
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 19:21:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/10 19:21:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

####
#### La base de datos ha sido limpiada y modificada a este punto
#### 
####
file_path = "/Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/DataModified/expanded_database_ecommerce"
df = spark.read.csv(file_path, header=True, inferSchema=True)

df.head()

/opt/anaconda3/envs/env-pyspark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
25/05/10 19:21:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

Row(event_time=datetime.date(2019, 11, 17), event_type='view', product_id=5300440, category_id=2053013563173241677, brand='vitek', price=17.76, user_id=513341639, user_session='d9544029-2739-4d16-9cac-79650460d9f0', event_time_ts=datetime.datetime(2019, 11, 17, 5, 35, 32), parent_category='None', subcategory='None', price_bucket='low', day_of_week=1, is_weekend=True)

In [4]:
df = df.selectExpr(
    'cast(event_time as timestamp) event_time',
    'cast(event_type as string) event_type',
    'cast(product_id as int) product_id',
    'cast(category_id as long) category_id',
    'cast(brand as string) brand',
    'cast(price as float) price',
    'cast(user_id as int) user_id',
    'cast(user_session as string) user_session',
    'cast(parent_category as string) parent_category',
    'cast(subcategory as string) subcategory',
    'cast(price_bucket as string) price_bucket',
    'cast(event_time_ts as timestamp) event_time_ts',
    'cast(day_of_week as int) day_of_week',
    'cast(is_weekend as boolean) is_weekend',
)

df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- parent_category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- price_bucket: string (nullable = true)
 |-- event_time_ts: timestamp (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- is_weekend: boolean (nullable = true)

